# <center>Sort

## [56. Merge Intervals (medium)](https://leetcode-cn.com/problems/merge-intervals/)

Given a collection of intervals, merge all overlapping intervals.

In [3]:
public int[][] merge(int[][] intervals) {
    // 首先对区间数组按起点从小到大进行排序，这样合并操作可以从左至右依序进行
    Arrays.sort(intervals, new Comparator<int[]>(){
        public int compare(int[] arr1, int[] arr2){
            return arr1[0] - arr2[0];
        }
    });
    
    // 思路一：用一个指针i遍历区间，若i+1的起点小于i的终点，就进行合并，直到i+1的起点大于i的终点
    List<int[]> res = new ArrayList<>();
    int i = 0;
    while(i < intervals.length){
        int left = intervals[i][0];   // 第i个区间的起点
        int right = intervals[i][1];  // 第i个区间的终点
        while(i < intervals.length - 1 && right >= intervals[i + 1][0]){ // 因为要取i+1，所以i < length-1
            i++;
            right = Math.max(right, intervals[i][1]);  // 更新终点，注意取最大值
        }
        // 直到i的终点(right)小于i+1的起点，将left到right合并
        res.add(new int[]{left, right});
        i++; //继续看下一个区间
    }
    return res.toArray(new int[res.size()][]);
}


public int[][] merge2(int[][] intervals) {
    // 首先对区间数组按起点从小到大进行排序，这样合并操作可以从左至右依序进行
    Arrays.sort(intervals, new Comparator<int[]>(){
        public int compare(int[] arr1, int[] arr2){
            return arr1[0] - arr2[0];
        }
    });
    
    // 思路二：对于当前区间，依次看它后面的区间能否合并
    List<int[]> res = new ArrayList<>();
    Set<Integer> merged = new HashSet<>(); // 用来保存已合并过的区间，合并过的区间不再重复访问
    for(int i = 0; i < intervals.length; i++){
        if(merged.contains(i)) continue;
        int[] cur = intervals[i];
        for(int j = i + 1; j < intervals.length; j++){ //因为已排序，所以从i+1开始看
            if(intervals[j][0] <= cur[1]){
                cur[1] = Math.max(cur[1], intervals[j][1]);
                merged.add(j);
            } else break;
        }
        res.add(cur);
    }
    return res.toArray(new int[res.size()][]);
}

## [剑指Offer 40. 最小的k个数 (easy)](https://leetcode-cn.com/problems/zui-xiao-de-kge-shu-lcof/)

输入整数数组 ```arr``` ，找出其中最小的 ```k``` 个数。例如，输入4、5、1、6、2、7、3、8这8个数字，则最小的4个数字是1、2、3、4。

<font color='dd0000'>

**Top K**类问题，3种解法：

1. 快排(最优)：不需要对整个数组进行排序，只需要根据交换点的位置决定下一轮排哪一半，当交换点正好是*K-1*时就可以结束（结果可能无序）；


2. 堆：维护一个大小为*K*的堆，堆中始终保持最小或最大的*K*个数。Java中可用优先队列来创建堆；


3. BST：（只能用于Top K个最小的数）和堆思想类似，维护大小为*K*的BST，始终保持最小的*K*个数。Java中用TreeMap创建。

</font>

In [11]:
// 方法1：快排
public int[] getLeastNumbers11(int[] arr, int k) {
    quickSort(arr, 0, arr.length - 1, k);
    int[] res = new int[k];
    for(int i = 0; i < k; i++){
        res[i] = arr[i];
    }
    return res;
}

private void quickSort(int[] arr, int left, int right, int k){
    int i = left;
    int j = right;
    int baseIndex = (int) (Math.random() * (right - left)) + left; // !!!随机选择基准点
    swap(arr, left, baseIndex);
    int base = arr[left];
    while(i != j){
        while(j > i && arr[j] >= base) j--;
        while(i < j && arr[i] <= base) i++;
        swap(arr, i, j);
    }
    swap(arr, left, i);
    if(i == k - 1) return;  // 如果交换点就是k-1位置，它左边一定都是比它小的，就不需要再排了，直接返回
    if(i > k - 1) quickSort(arr, left, i - 1, k);  // 如果交换点 > k-1，就再对左边排
    if(i < k - 1) quickSort(arr, i + 1, right, k); // 如果交换点 < k-1，就再对右边排
}

private void swap(int[] arr, int i, int j){
    int t = arr[i];
    arr[i] = arr[j];
    arr[j] = t;
}



// 另一种写法：每次用快排切分返回交换点索引，再对索引进行判断
public int[] getLeastNumbers12(int[] arr, int k) {
    if(arr.length < 1 || k == 0)
        return new int[0];

    return helper(arr, 0, arr.length - 1, k);        
}

private int[] helper(int[] arr, int left, int right, int k){
    int index = quickPartition(arr, left, right);
    if(index == k - 1) return Arrays.copyOf(arr, k);
    return index > (k - 1) ? helper(arr, left, index - 1, k) : helper(arr, index + 1, right, k);
}

// 返回的是交换点的索引
private int quickPartition(int[] arr, int left, int right){
    int i = left;
    int j = right;
    int base = arr[left];
    while(i != j){
        while(j > i && arr[j] >= base) j--;
        while(i < j && arr[i] <= base) i++;
        int temp = arr[i];
        arr[i] = arr[j];
        arr[j] = temp;
    }
    arr[left] = arr[i];
    arr[i] = base;
    return i;
}

In [12]:
// 方法2：堆
public int[] getLeastNumbers2(int[] arr, int k) {
    if(arr.length < 1 || k == 0)
        return new int[0];
        
    // 默认的是小顶堆，但是本题要求最小的K个数，遍历时要让大的数出队，所以用比较器创建大顶堆
    Queue<Integer> heap = new PriorityQueue<>(new Comparator<Integer>(){
        public int compare(Integer a1, Integer a2){
            return a2 - a1;
        }
    });
    for(int i = 0; i < arr.length; i++){
        if(heap.size() < k){
            heap.offer(arr[i]);
        }
        else{
            if(arr[i] < heap.peek()){ // 当前值更小，就把大的数弹出，当前数进队
                heap.poll();
                heap.offer(arr[i]);
            }
        }
    }
    int[] res = new int[k];
    for(int i = 0; i < k; i++){
        res[i] = heap.poll();
    }
    return res;
}

In [13]:
// 方法3：BST
public int[] getLeastNumbers2(int[] arr, int k) {
    if(arr.length < 1 || k == 0)
        return new int[0];
    
    // TreeMap中，key是数组中的数字，value是该数字的个数。总个数不能超过k
    TreeMap<Integer, Integer> BST = new TreeMap<>();
    int count = 0;
    for(int a : arr){
        if(count < k){
            BST.put(a, BST.getOrDefault(a, 0) + 1);  // 注意 getOrDefault(key, defaultValue)方法的使用
            count++;
        } else{
            Map.Entry<Integer, Integer> biggest = BST.lastEntry(); // 取出最大的数，即BST中的最后一个
            if(a < biggest.getKey()){
                BST.put(a, BST.getOrDefault(a, 0) + 1);
                if(biggest.getValue() == 1) BST.pollLastEntry();
                else BST.put(biggest.getKey(), biggest.getValue() - 1);
            }
        }
    }
    int[] res = new int[k];
    int index = 0;
    for(Map.Entry<Integer, Integer> entry : BST.entrySet()){
        int n = entry.getValue();
        while(n-- != 0){
            res[index++] = entry.getKey();
        }
    }
    return res;
}

## [215. Kth Largest Element in an Array (medium)](https://leetcode-cn.com/problems/kth-largest-element-in-an-array/)

Find the **k**th largest element in an unsorted array. Note that it is the kth largest element in the sorted order, not the kth distinct element.

<font color='dd0000'>和上题同样的思路，不过是求**第*K*大**的。下面只写出快排解法</font>

In [14]:
public int findKthLargest(int[] nums, int k) {
    quickSort(nums, 0, nums.length - 1, k);
    return nums[k - 1];
}
    
private void quickSort(int[] nums, int left, int right, int k){
    int i = left;
    int j = right;
    int baseIndex = (int) (Math.random() * (right - left) + left);
    int base = nums[baseIndex]; //随机选择基准值
    nums[baseIndex] = nums[left];
    nums[left] = base;
    while(i != j){
        while( j > i && nums[j] <= base) j--;
        while( i < j && nums[i] >= base) i++;
        int temp = nums[i];
        nums[i] = nums[j];
        nums[j] = temp;
    }
    nums[left] = nums[i];
    nums[i] = base;
    if(i == k - 1) return;
    if(i > k - 1) quickSort(nums, left, i - 1, k);
    if(i < k - 1) quickSort(nums, i + 1, right, k);
}